In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-data/train.csv


# Whatever I have to do(pseudo-code)

* Import All Required Libraries
* Load Dataset
* Basic Data Overview (head, shape, dtypes)
* Check Missing Values
* Split Features & Target
* Identify Numeric & Categorical Columns
* Create Numeric Preprocessing Steps
* Create Categorical Preprocessing Steps
* Build ColumnTransformer
* Create ML Pipeline (Preprocess + Logistic Regression)
* Train-Test Split
* Fit the Model
* Evaluate the Model (Accuracy/Score)
* Optional: Improve Model with GridSearchCV

In [2]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression

In [3]:
df= pd.read_csv('/kaggle/input/train-data/train.csv')

In [4]:
df.tail()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [7]:
df.shape

(891, 8)

In [8]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [9]:
X= df.drop(columns= 'Survived')
y= df['Survived']

In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 42)

In [11]:
X_train.shape

(712, 7)

In [12]:
y_train

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

In [13]:
numerical_feature= ['Age', 'Fare']
numerical_transform= Pipeline(steps= [
    ('imputer', SimpleImputer(strategy= 'median')),
    ('scaler', StandardScaler())
])

categorical_feature= ['Embarked', 'Sex']
categorical_transform= Pipeline(steps= [
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown= 'ignore'))
])

In [14]:
processing= ColumnTransformer(transformers=[
    ('num', numerical_transform, numerical_feature),
    ('cat', categorical_transform, categorical_feature)
], remainder= 'passthrough')

In [15]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [16]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.barplot(x='Age', y='Fare', data=df)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
clf_model= Pipeline(steps=[
    ('processing', processing),
    ('classifier', LogisticRegression())
])

In [19]:
from sklearn import set_config

set_config(display='diagram')
clf_model

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [20]:
param_grid= {
    'processing__num__imputer__strategy': ['mean', 'median'],
    'processing__cat__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search= GridSearchCV(clf_model, param_grid, cv= 10)

In [21]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [22]:
grid_search.fit(X_train, y_train)

print(f'Best params: ')
print(grid_search.best_params_)

Best params: 
{'classifier__C': 1.0, 'processing__cat__imputer__strategy': 'constant', 'processing__num__imputer__strategy': 'median'}


In [23]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.796


In [24]:
import pandas as pd

cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_classifier__C','param_processing__cat__imputer__strategy','param_processing__num__imputer__strategy','mean_test_score']]

,param_classifier__C,param_processing__cat__imputer__strategy,param_processing__num__imputer__strategy,mean_test_score
7,1.0,constant,median,0.796283
8,10,most_frequent,mean,0.796283
10,10,constant,mean,0.796283
12,100,most_frequent,mean,0.796283
14,100,constant,mean,0.796283
3,0.1,constant,median,0.794894
4,1.0,most_frequent,mean,0.794875
5,1.0,most_frequent,median,0.794875
6,1.0,constant,mean,0.794875
9,10,most_frequent,median,0.794875
